# Example of using ChatGPT API for article summarization
Refer to: https://colab.research.google.com/drive/15jh4v_TBPsTyIBhi0Fz46gEkjvhzGaBR?usp=sharing


In [1]:
# If needed in Colab:
# !pip -q install openai==1.43.0 gradio==4.44.0

import os
import gradio as gr
from openai import OpenAI

# --- API key: use Colab Secrets (preferred) or env var ---
OPENAI_API_KEY = None
try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
except Exception:
    pass
if not OPENAI_API_KEY:
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise RuntimeError("Set OPENAI_API_KEY in Colab Secrets or env.")

client = OpenAI(api_key=OPENAI_API_KEY)

MODEL = "gpt-4o-mini"   # you can switch to "gpt-4o"
SYSTEM_PROMPT = (
    "You are a helpful, concise assistant. "
    "Always use the full conversation context to avoid repeating information."
)

def chat(user_input, chatbot, temp, max_toks):
    # Build message list from prior turns so the model has context
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]
    for input_text, response_text in chatbot:
        if input_text:
            messages.append({"role": "user", "content": input_text})
        if response_text:
            messages.append({"role": "assistant", "content": response_text})

    # Add the new user turn
    messages.append({"role": "user", "content": user_input})

    # Call Chat Completions API with full history
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=temp,
        max_tokens=max_toks,
    )

    assistant_reply = response.choices[0].message.content

    # Update the visible chat history so Gradio shows all turns
    chatbot = chatbot + [[user_input, assistant_reply]]
    # Return "" to clear the textbox, and updated chatbot history
    return "", chatbot

# ----------------- Gradio UI -----------------
with gr.Blocks(title="Context-Aware ChatGPT") as demo:
    gr.Markdown("### Chatbot with Conversation History\n"
                "All previous turns are sent to the API so the model keeps context.")
    chatbot_ui = gr.Chatbot(height=420, show_copy_button=True)

    with gr.Row():
        msg = gr.Textbox(placeholder="Type your message…", label="Your message", scale=4)
        send = gr.Button("Send", scale=1)

    with gr.Row():
        temp = gr.Slider(0.0, 1.0, value=0.5, step=0.1, label="Temperature")
        max_toks = gr.Slider(64, 512, value=200, step=16, label="Max tokens")

    clear = gr.Button("Clear history")

    # Wire up events: both Enter and Send use the same chat() function
    msg.submit(chat, inputs=[msg, chatbot_ui, temp, max_toks], outputs=[msg, chatbot_ui])
    send.click(chat, inputs=[msg, chatbot_ui, temp, max_toks], outputs=[msg, chatbot_ui])
    clear.click(lambda: None, None, chatbot_ui, queue=False)

demo.launch()


/tmp/ipython-input-122231703.py:59: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_ui = gr.Chatbot(height=420, show_copy_button=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ed3a997c621a15e333.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
